In [1]:
!python -V

Python 3.7.4


In [2]:
import pandas as pd

In [3]:
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [30]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")


In [23]:
mlflow.__version__

'1.26.1'

In [9]:
mlflow.set_experiment("nyc-taxi-experiment_v1")

2022/05/30 22:51:32 INFO mlflow.tracking.fluent: Experiment with name 'nyc-taxi-experiment_v1' does not exist. Creating a new experiment.


<Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='nyc-taxi-experiment_v1', tags={}>

In [10]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [12]:
df_train = read_dataframe('./green_tripdata_2021-01.parquet')
df_val = read_dataframe('./green_tripdata_2021-02.parquet')

In [13]:
len(df_train), len(df_val)

(73908, 61921)

In [14]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [15]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [16]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [18]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.479614645724615

In [20]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [21]:
lr = Lasso(0.01)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.167275941179728

In [24]:
with mlflow.start_run():

    mlflow.set_tag("developer", "Vishnu")

    mlflow.log_param("train-data-path", "./green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./green_tripdata_2021-02.csv")

    alpha = 0.5
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [25]:
import xgboost as xgb

In [26]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [27]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [28]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [29]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[21:13:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:11.19163                                                       
[1]	validation-rmse:7.91436                                                        
[2]	validation-rmse:6.98446                                                        
[3]	validation-rmse:6.70410                                                        
[4]	validation-rmse:6.60035                                                        
[5]	validation-rmse:6.55881                                                        
[6]	validation-rmse:6.53751                                                        
[7]	validation-rmse:6.52555                                                        
[8]	validation-rmse:6.51954                                                        
[9]	validation-rmse:6.51476                                        

[95]	validation-rmse:6.42738                                                       
[96]	validation-rmse:6.42905                                                       
[97]	validation-rmse:6.42934                                                       
[98]	validation-rmse:6.43002                                                       
[99]	validation-rmse:6.42915                                                       
[100]	validation-rmse:6.42945                                                      
[101]	validation-rmse:6.42909                                                      
[102]	validation-rmse:6.42906                                                      
[103]	validation-rmse:6.42885                                                      
[104]	validation-rmse:6.42862                                                      
[105]	validation-rmse:6.42791                                                      
[106]	validation-rmse:6.42739                                               

[78]	validation-rmse:6.42249                                                       
[79]	validation-rmse:6.42211                                                       
[80]	validation-rmse:6.42179                                                       
[81]	validation-rmse:6.42134                                                       
[82]	validation-rmse:6.42047                                                       
[83]	validation-rmse:6.41990                                                       
[84]	validation-rmse:6.41909                                                       
[85]	validation-rmse:6.41854                                                       
[86]	validation-rmse:6.41790                                                       
[87]	validation-rmse:6.41754                                                       
[88]	validation-rmse:6.41716                                                       
[89]	validation-rmse:6.41667                                                

[175]	validation-rmse:6.39441                                                      
[176]	validation-rmse:6.39437                                                      
[177]	validation-rmse:6.39406                                                      
[178]	validation-rmse:6.39413                                                      
[179]	validation-rmse:6.39435                                                      
[180]	validation-rmse:6.39444                                                      
[181]	validation-rmse:6.39404                                                      
[182]	validation-rmse:6.39411                                                      
[183]	validation-rmse:6.39370                                                      
[184]	validation-rmse:6.39369                                                      
[185]	validation-rmse:6.39411                                                      
[186]	validation-rmse:6.39403                                               

[14]	validation-rmse:6.75648                                                       
[15]	validation-rmse:6.73246                                                       
[16]	validation-rmse:6.71351                                                       
[17]	validation-rmse:6.69928                                                       
[18]	validation-rmse:6.68681                                                       
[19]	validation-rmse:6.68015                                                       
[20]	validation-rmse:6.67220                                                       
[21]	validation-rmse:6.66733                                                       
[22]	validation-rmse:6.66360                                                       
[23]	validation-rmse:6.66044                                                       
[24]	validation-rmse:6.65687                                                       
[25]	validation-rmse:6.65419                                                

...

[723]	validation-rmse:6.46710                                                      
[724]	validation-rmse:6.46708                                                      
[725]	validation-rmse:6.46696                                                      
[726]	validation-rmse:6.46693                                                      
[727]	validation-rmse:6.46681                                                      
[728]	validation-rmse:6.46676                                                      
[729]	validation-rmse:6.46668                                                      
[730]	validation-rmse:6.46652                                                      
[731]	validation-rmse:6.46646                                                      
[732]	validation-rmse:6.46618                                                      
[733]	validation-rmse:6.46600                                                      
[734]	validation-rmse:6.46616                                               

[820]	validation-rmse:6.45716                                                      
[821]	validation-rmse:6.45706                                                      
[822]	validation-rmse:6.45703                                                      
[823]	validation-rmse:6.45679                                                      
[824]	validation-rmse:6.45674                                                      
[825]	validation-rmse:6.45659                                                      
[826]	validation-rmse:6.45660                                                      
[827]	validation-rmse:6.45654                                                      
[828]	validation-rmse:6.45649                                                      
[829]	validation-rmse:6.45623                                                      
[830]	validation-rmse:6.45611                                                      
[831]	validation-rmse:6.45613                                               

[917]	validation-rmse:6.44803                                                      
[918]	validation-rmse:6.44803                                                      
[919]	validation-rmse:6.44794                                                      
[920]	validation-rmse:6.44819                                                      
[921]	validation-rmse:6.44818                                                      
[922]	validation-rmse:6.44816                                                      
[923]	validation-rmse:6.44805                                                      
[924]	validation-rmse:6.44780                                                      
[925]	validation-rmse:6.44772                                                      
[926]	validation-rmse:6.44780                                                      
[927]	validation-rmse:6.44773                                                      
[928]	validation-rmse:6.44754                                               

[12]	validation-rmse:10.79762                                                      
[13]	validation-rmse:10.38781                                                      
[14]	validation-rmse:10.02027                                                      
[15]	validation-rmse:9.68532                                                       
[16]	validation-rmse:9.37810                                                       
[17]	validation-rmse:9.10198                                                       
[18]	validation-rmse:8.85476                                                       
[19]	validation-rmse:8.62650                                                       
[20]	validation-rmse:8.41838                                                       
[21]	validation-rmse:8.23535                                                       
[22]	validation-rmse:8.07296                                                       
[23]	validation-rmse:7.92275                                                

KeyboardInterrupt: 

In [18]:
mlflow.xgboost.autolog(disable=True)

In [21]:
with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[11:40:58] WARNING: /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-11.0-arm64-3.8/xgboost/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95634
[2]	validation-rmse:16.59114
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32800
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47583
[10]	validation-rmse:9.96483
[11]	validation-rmse:9.51784
[12]	validation-rmse:9.12620
[13]	validation-rmse:8.78522
[14]	validation-rmse:8.49037
[15]	validation-rmse:8.23332
[16]	validation-rmse:8.01179
[17]	validation-rmse:7.81822
[18]	validation-rmse:7.65209
[19]	validation-rmse:7.50743
[20]	validation-rmse:7.38235
[21]	validation-rmse:7.27382
[22]	validation-rmse:7.17865
[23]	validation-rmse:7.09606
[24]	validation-rmse:7.02481
[25]	validation-rmse:6.96336
[26]	validation-rmse:6.91001
[27]	valid

In [23]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/Users/cristian.martinez/miniconda3/envs/exp-tracking-env/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


  File "hypo.py", line 27
    with mlflow.start_run()
                          ^
SyntaxError: invalid syntax
